In [10]:
import tempfile
import os

import pandas as pd

import tensorflow as tf

In [11]:
!pip install -q -U tensorflow_datasets

In [12]:
import tensorflow_datasets as tfds

In [13]:
CSV_COLUMNS =  ['age','gender','polyuria','polydipsia','sudden_weight_loss', 'weakness', 'polyphagia', 'genital_thrush', 'visual_blurring','itching','irritability', 'delayed_healing', 'partial_paresis', 'muscle_stiffness', 'alopecia', 'obesity', 'cls']


In [14]:
def train_input_fn():
  diabete = tf.data.experimental.make_csv_dataset(
      "diabetes_data_upload.csv",
      batch_size=32,
      header=True,
      column_names=CSV_COLUMNS,
      label_name="cls")
  diabete_batches = (
      diabete.cache().repeat().shuffle(500)
      .prefetch(tf.data.AUTOTUNE))
  return diabete_batches

In [15]:
def categorical_fc(name, values):
    """Helper function to wrap categorical feature by indicator column.

    Args:
        name: str, name of feature.
        values: list, list of strings of categorical values.
    Returns:
        Indicator column of categorical feature.
    """
    cat_column = tf.feature_column.categorical_column_with_vocabulary_list(
            key=name, vocabulary_list=values)

    return tf.feature_column.indicator_column(categorical_column=cat_column)

def create_feature_columns():
    """Creates dictionary of feature columns from inputs.

    Returns:
        Dictionary of feature columns.
    """
    feature_columns = {
        colname : tf.feature_column.numeric_column(key=colname)
           for colname in ["age"]
    }

    feature_columns["gender"] = categorical_fc("gender", ['Male', 'Female'])
    
    for colname in ['gender','polyuria','polydipsia','sudden_weight_loss', 'weakness', 'polyphagia', 'genital_thrush', 'visual_blurring','itching','irritability', 'delayed_healing', 'partial_paresis', 'muscle_stiffness', 'alopecia', 'obesity']:        
        feature_columns[colname] = categorical_fc(colname, ['Yes', 'No'])

    return feature_columns

In [16]:
fc = create_feature_columns()
for key,val in fc.items():
    exec(key + '=val')

In [19]:
model_dir = tempfile.mkdtemp()
model = tf.estimator.DNNClassifier(
    model_dir=model_dir,
    hidden_units=[128,80,50,30],
    feature_columns=[age,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,itching,irritability,delayed_healing,partial_paresis,muscle_stiffness,alopecia,obesity],
    n_classes=2,
    label_vocabulary=['Positive','Negative'],
    activation_fn='relu',
    optimizer='Adam'
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpvrpeyrv8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
model = model.train(input_fn=train_input_fn, steps=8000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpvrpeyrv8/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.672667, step = 0
INFO:tensorflow:global_step/sec: 123.976
INFO:tensorflow:loss = 0.32409817, step = 100 (0.808 sec)
INFO:tensorflow:global_step/sec: 165.362
INFO:tensorflow:loss = 0.167272, step = 200 (0.605 sec)
INFO:tensorflow:global_step/sec: 147.159
INFO:tensorflow:loss = 0.14353347, step = 300 (0.680 sec)
INFO:tensorflow:global_step/sec: 161.784
INFO:tensorflow:loss = 0.12913913, step = 400 (0.618 sec)
INFO:tensorflow:global_step/sec: 144.639
INFO:tensorflow:loss = 0.3744747, step = 500 (0.692 sec)
INFO:

In [21]:
result = model.evaluate(train_input_fn, steps=10)

for key, value in result.items():
  print(key, ":", value)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-19T10:18:20Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvrpeyrv8/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Inference Time : 1.86874s
INFO:tensorflow:Finished evaluation at 2021-01-19-10:18:22
INFO:tensorflow:Saving dict for global step 8000: accuracy = 0.978125, accuracy_baseline = 0.603125, auc = 0.999082, auc_precision_recall = 0.9986322, average_loss = 0.041661553, global_step = 8000, label/mean = 0.396875, loss = 0

In [22]:
for pred in model.predict(train_input_fn):
  for key, value in pred.items():
    print(key, ":", value)
  break

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvrpeyrv8/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
logits : [-14.877044]
logistic : [3.4592506e-07]
probabilities : [9.9999964e-01 3.4592495e-07]
class_ids : [0]
classes : [b'Positive']
all_class_ids : [0 1]
all_classes : [b'Positive' b'Negative']
